# Tutorial 1. Network Generation, part 1

Netsim package includes several modules:
- Netgen (network generator). This module includes several functions aimed at generating different network configurations. This module is the subject of this notebook.
- Netsim (network simulation). 

The *NetSim* simulation uses information stored in a (attribute) table to set up the simulation. The table must contain a series columns with concrete header names (see below). Values within these columns must be restricted to specific ranges. 
- *id*: exclusive identifier for each location. 
- *group*: identifies a location as being part of a specific group. Groups can be of any size. Groups of size 1 will  automatically mixed (see below) with the following groups. A column with a single group affiliation will be created in case this column does not exist. *Default*:1.
- *seq*: identifies rank/ordering of location within a group. There are two possible options:
  - *No ordering/ ranking (default)*. Within each group use a **single** value for all locations in that group if you do not want to identify any particular ranking or ordering within the group. Depending on the number of locations in the group, the *NetSim* with either generate all possible permutations or a particular fixed number, *num_samples* of randomized samples (with repetition).
  - *Ordering /ranking*. To specify the order in which locations are going to join a network, use any sequence of numbers (with no repetitions) for all locations in a group.
- *mix*: indicates whether any location from a group can mix with a location from the previous group. This value must be the same for all members in a group (either 0 or 1). *Default*:1. *n.b.* The mix paramater for all locations in the first group is always set to 0



- residuality index (ri). This is a fraction $\ge$ 0 and $\le$ 1.0. *Default*: $\frac{1}{num \ in \ group}$ for all sites in a group. It must be the same for all members in a group.



If any of these columns are not present the simulation will genereate these and populate them with default values. 

### Imports

In [1]:
import geopandas as gpd
import netsim.netgen as ng

### Read a sample shapefile

In [2]:
fn = r'..\data\sample\sample5.shp'

We use geopandas to read the shapefile that contains the location and the columns needed to run the simulation. In case the location file is a simple a text file, e.g. *comma-delimited* or *csv*, use
```python
import pandas as pd
df = pd.read_csv(filename)
```
instead.


In [3]:
df= gpd.read_file(fn)

In [4]:
df

,id,seq,group,mix,easting,northing,geometry
0,0,1,1,0,530782,4389390,POINT (530782 4389390)
1,1,1,1,0,531119,4388860,POINT (531119 4388860)
2,2,1,1,0,530403,4388580,POINT (530403 4388580)
3,3,1,1,0,530503,4388620,POINT (530503 4388620)
4,4,1,1,0,530729,4388930,POINT (530729 4388930)
5,5,1,1,0,530606,4389150,POINT (530606 4389150)


**N.B.** It is a good idea to make a copy of your original table in case we need to change values along the way

### Checking values

We run the ```setup()``` function in the *netgen* module to check whether the table with the information needed to run the simulation has all the appropriate columns and values within these. 

In [5]:
df = ng.setup(df)


 No corrections or errors !! 


Let's introduce some errors and see how the ```setup()``` function behaves. Let's remove the 'mix' column.

In [6]:
df.drop(columns=['mix'], inplace= True)

In [7]:
df = ng.setup(df)
df

,id,seq,group,easting,northing,geometry,mix
0,0,1,1,530782,4389390,POINT (530782 4389390),0
1,1,1,1,531119,4388860,POINT (531119 4388860),0
2,2,1,1,530403,4388580,POINT (530403 4388580),0
3,3,1,1,530503,4388620,POINT (530503 4388620),0
4,4,1,1,530729,4388930,POINT (530729 4388930),0
5,5,1,1,530606,4389150,POINT (530606 4389150),0


The sequence ('seq') column must contain a value of '1' or a montonic sequence of numbers per group. Let's change it so that the sequence we have is no longer monotonic 

In [8]:
df.loc[3,'seq']= 2
df

,id,seq,group,easting,northing,geometry,mix
0,0,1,1,530782,4389390,POINT (530782 4389390),0
1,1,1,1,531119,4388860,POINT (531119 4388860),0
2,2,1,1,530403,4388580,POINT (530403 4388580),0
3,3,2,1,530503,4388620,POINT (530503 4388620),0
4,4,1,1,530729,4388930,POINT (530729 4388930),0
5,5,1,1,530606,4389150,POINT (530606 4389150),0


Now, when we run ```setup()``` function it will spit out an error

In [15]:
ng.setup(df)


 No corrections or errors !! 


,id,seq,group,easting,northing,geometry,mix
0,0,3,1,530782,4389390,POINT (530782 4389390),0
1,1,4,1,531119,4388860,POINT (531119 4388860),0
2,2,5,1,530403,4388580,POINT (530403 4388580),0
3,3,6,1,530503,4388620,POINT (530503 4388620),0
4,4,7,1,530729,4388930,POINT (530729 4388930),0
5,5,8,1,530606,4389150,POINT (530606 4389150),0


We can replace the original 'seq' column by another one. Provided all values in a group are unique the simulation will work. 

In [16]:
df['seq']= [3,4,5,6,7,8]
ng.setup(df)

,id,seq,group,easting,northing,geometry,mix
0,0,3,1,530782,4389390,POINT (530782 4389390),0
1,1,4,1,531119,4388860,POINT (531119 4388860),0
2,2,6,1,530403,4388580,POINT (530403 4388580),0
3,3,5,1,530503,4388620,POINT (530503 4388620),0
4,4,7,1,530729,4388930,POINT (530729 4388930),0
5,5,8,1,530606,4389150,POINT (530606 4389150),0


### Creating a network generator

The next step is to run the ```create_network_generator()``` function. The main aim of this function is to generate a **network generator** that we can later use to produce different versions, or iterations, of our network. The function provide us with additional information: a dictionary with details about the type of iteration for each group and the total number of iterations that are possible. Let's explore this function

In [18]:
netgentor, net_info, total_iterations = ng.create_network_generator(df)


 iteration broken per group....

   group iter_type  num_iter  num_loc
0      1    single         1        6

 total number of iterations.... 1


In [19]:
net_info

,group,iter_type,num_iter,num_loc
0,1,single,1,6


The above example is not very informative given that we have specified one one group with a single ordering. Let us change the sequence column to all 1s and see what happens.

In [21]:
df['seq'] = [1,1,1,1,1,1]
df

,id,seq,group,easting,northing,geometry,mix,seg
0,0,1,1,530782,4389390,POINT (530782 4389390),0,1
1,1,1,1,531119,4388860,POINT (531119 4388860),0,1
2,2,1,1,530403,4388580,POINT (530403 4388580),0,1
3,3,1,1,530503,4388620,POINT (530503 4388620),0,1
4,4,1,1,530729,4388930,POINT (530729 4388930),0,1
5,5,1,1,530606,4389150,POINT (530606 4389150),0,1


In [22]:
netgentor, net_info, total_iterations = ng.create_network_generator(df)


 iteration broken per group....

   group iter_type  num_iter  num_loc
0      1    sample       100        6

 total number of iterations.... 100


In [23]:
net_info

,group,iter_type,num_iter,num_loc
0,1,sample,100,6


Notice how the iteration type (iter_type) has changed from 'single' to 'sample' and the number of iterations (num_iter) has gone from 1 to 100 (also the total number of iterations is now 100). This number 100 is the default number of interations that are generated when the number of locations in a group is greater than 5. 

Here is an example of one of the possible network iterations,

In [25]:
list(next(netgentor))

[(4, 2, 3, 0, 1, 5)]

You can use ```next()``` function on the network generator instance repeatedly until you arrive to *total_iterations*. For instance, the next code generates 5 new interations,

In [26]:
for i in range(5):
    print(next(netgentor))

[(3, 1, 0, 2, 4, 5)]
[(5, 1, 0, 4, 3, 2)]
[(4, 0, 3, 1, 5, 2)]
[(1, 3, 5, 2, 0, 4)]
[(2, 4, 3, 5, 0, 1)]


Let's change the number of groups so that we end up with two groups, of two and four locations.

In [31]:
sel = df['id'].isin([2, 3, 4, 5])
df.loc[sel,'group']= 2
df

,id,seq,group,easting,northing,geometry,mix,seg
0,0,1,1,530782,4389390,POINT (530782 4389390),0,1
1,1,1,1,531119,4388860,POINT (531119 4388860),0,1
2,2,1,2,530403,4388580,POINT (530403 4388580),0,1
3,3,1,2,530503,4388620,POINT (530503 4388620),0,1
4,4,1,2,530729,4388930,POINT (530729 4388930),0,1
5,5,1,2,530606,4389150,POINT (530606 4389150),0,1


Let's first re-run ```setup()``` to make sure that our new dataframe is fine and then ```create_network_generator()```

In [32]:
ng.setup(df)


 No corrections or errors !! 


,id,seq,group,easting,northing,geometry,mix,seg
0,0,1,1,530782,4389390,POINT (530782 4389390),0,1
1,1,1,1,531119,4388860,POINT (531119 4388860),0,1
2,2,1,2,530403,4388580,POINT (530403 4388580),0,1
3,3,1,2,530503,4388620,POINT (530503 4388620),0,1
4,4,1,2,530729,4388930,POINT (530729 4388930),0,1
5,5,1,2,530606,4389150,POINT (530606 4389150),0,1


In [33]:
netgentor, net_info, total_iterations = ng.create_network_generator(df)


 iteration broken per group....

   group    iter_type  num_iter  num_loc
0      1  permutation         2        2
1      2  permutation        24        4

 total number of iterations.... 48


Notice how the iteration type (iter_type) has changed to 'permutation' and how the total number of permutations (48) is the product of two permutations. Let's ask for a few iterations and see what they look like,

In [34]:
for i in range(5):
    print(next(netgentor))

((0, 1), (2, 3, 4, 5))
((0, 1), (2, 3, 5, 4))
((0, 1), (2, 4, 3, 5))
((0, 1), (2, 4, 5, 3))
((0, 1), (2, 5, 3, 4))


Next, given a single network iteration, we call the ```network_layout()``` function, that uses the **mix** parameter to determine the ordering of paths. This function expects the *corrected* original dataframe (containing all locations that make-up the network), an outputs a network dataframe with 'origin', destination' and 'iteration' fields, and the iteration number. It will return the output network dataframe with the origin-destination and iteration number for each path in the network. 

##### Create an empty df

In [54]:
df_out_net = pd.DataFrame(columns=['origin', 'destination', 'iteration'])
df_out_net['iteration'] = pd.to_numeric(df_out_net['iteration'], downcast='integer')

In [55]:
num_iterations = 5

In [56]:
for i in range(num_iterations):
    sample_iteration = list(next(netgentor))
    df_out_net = network_layout(df, sample_iteration, i, df_out_net, option='expanded')

In [57]:
df_out_net

,origin,destination,iteration
0,0,1,0
1,5,0,0
2,5,1,0
3,3,0,0
4,3,1,0
5,3,5,0
6,2,0,0
7,2,1,0
8,2,5,0
9,2,3,0


In [146]:
gmax= 4
gt = 0
T = 12

In [147]:
for i in range(100):
    gt= gt - gt/T + (1 - gt/gmax)
    print(gt)

1.0
1.6666666666666665
2.111111111111111
2.4074074074074074
2.6049382716049383
2.736625514403292
2.824417009602195
2.88294467306813
2.9219631153787535
2.947975410252502
2.965316940168335
2.9768779601122235
2.984585306741482
2.989723537827655
2.993149025218437
2.995432683478958
2.996955122319305
2.9979700815462036
2.9986467210308025
2.9990978140205353
2.999398542680357
2.9995990284535714
2.9997326856357143
2.99982179042381
2.999881193615873
2.9999207957439156
2.9999471971626104
2.9999647981084068
2.9999765320722713
2.9999843547148477
2.9999895698098986
2.9999930465399327
2.9999953643599553
2.9999969095733032
2.9999979397155356
2.9999986264770238
2.9999990843180155
2.9999993895453434
2.9999995930302292
2.999999728686819
2.999999819124546
2.9999998794163636
2.999999919610909
2.999999946407273
2.999999964271515
2.99999997618101
2.9999999841206733
2.999999989413782
2.9999999929425214
2.9999999952950143
2.999999996863343
2.9999999979088954
2.9999999986059303
2.99999999907062
2.99999999938041

In [101]:
for i in range(10):
    gt= gt_1 - gt_1/T + (1 - gt_1/gmax)
    print(gt)
    gt_1 = gt

-124923.47242103516
-206124.529494708
-340106.2736662682
-561176.1515493426
-925941.4500564152
-1527804.192593085
-2520877.7177785905
-4159449.0343346745
-6863091.706652213
-11324102.115976151
-18684769.291360646
-30829870.13074507
-50869286.51572937
-83934323.55095346
-138491634.6590732
-228511197.98747078
-377043477.4793268
-622121738.6408893
-1026500869.5574672
-1693726435.569821
